In [1]:
%pip install -q --upgrade trl transformers peft accelerate bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.10.10 requires trl!=0.19.0,<=0.23.0,>=0.18.2, but you have trl 0.24.0 which is incompatible.
unsloth 2025.10.9 requires transformers!=4.52.0,!=4.52.1,!=4.52.2,!=4.52.3,!=4.53.0,!=4.54.0,!=4.55.0,!=4.55.1,<=4.56.2,>=4.51.3, but you have transformers 4.57.1 which is incompatible.
unsloth 2025.10.9 requires trl!=0.19.0,<=0.23.0,>=0.18.2, but you have trl 0.24.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, Dataset
from trl import SFTTrainer, SFTConfig
import math
from tqdm import tqdm
from peft import get_peft_model
import os

In [2]:
MODEL_NAME = "Qwen/Qwen3-0.6B"
MAX_LENGTH = 8192         # set according to model/context available
# IGNORE_INDEX = -100
PER_DEVICE_BATCH_SIZE = 1
GRAD_ACCUM = 8

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, dtype=torch.bfloat16)
# model.unload()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
import torch
from peft import LoraConfig
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset

In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [13]:
dataset = load_dataset('json',
                       data_files = 'instruction_tuned_dataset_processed.jsonl', split='train'
                      )

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
# Ensure content is string
def ensure_text(example):
    conv = []
    for msg in example["conversations"]:
        content = msg.get("content", "")
        if not isinstance(content, str):
            content = str(content)
        conv.append({"role": str(msg.get("role", "user")), "content": content})
    return {"conversations": conv}

dataset = dataset.map(ensure_text, desc="Normalizing message content")

# Format into a single "text" column using the model's chat template
def format_conversation(example):
    conv = example["conversations"]
    text = tokenizer.apply_chat_template(
        conv,
        tokenize=False,
        add_generation_prompt=False,  # training on full chat
    )
    return {"text": text}

dataset = dataset.map(format_conversation, desc="Formatting conversations -> text")


Normalizing message content:   0%|          | 0/83990 [00:00<?, ? examples/s]

Formatting conversations -> text:   0%|          | 0/83990 [00:00<?, ? examples/s]

In [22]:
dataset = dataset.filter(lambda x: isinstance(x["text"], str) and len(x["text"].strip()) > 0)


Filter:   0%|          | 0/83990 [00:00<?, ? examples/s]

In [24]:
# SFT training args
training_args = SFTConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,  # 32 may OOM for 7B + LoRA; adjust to your GPU
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    save_strategy="steps",
    save_steps=500,
    logging_steps=10,
    warmup_ratio=0.03,
    group_by_length=True,
    run_name="cook_assistant_1",
    bf16=True,
    packing=False,
    dataset_text_field="text",           # <-- use the "text" column
    max_length=MAX_LENGTH,
    assistant_only_loss=False,            # <-- now OK: formatted conversational text
    eos_token="<|im_end|>",              # Qwen chat templates use <|im_end|>
    pad_token=tokenizer.pad_token,
)

# --- LoRA Config ---
lora_config = LoraConfig(
    r=4, # Increased rank for potentially better performance
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=None, # Target more modules
)
model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
)

trainer.train()

/opt/conda/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/83990 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/83990 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/83990 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,3.344700
20,3.839300
30,4.107700
40,4.361200
50,4.754000
60,3.256600
70,3.678900
80,3.821000
90,3.872500
100,4.022600


TrainOutput(global_step=2625, training_loss=1.6465196068173362, metrics={'train_runtime': 2063.5954, 'train_samples_per_second': 40.701, 'train_steps_per_second': 1.272, 'total_flos': 3.9256519213056e+16, 'train_loss': 1.6465196068173362, 'entropy': 1.2175945169047306, 'num_tokens': 14507160.0, 'mean_token_accuracy': 0.7481753543803566, 'epoch': 1.0})

In [32]:
# 1) SAVE THE ADAPTER (PEFT) CORRECTLY
from peft import PeftModel
import os

ADAPTER_DIR = "./results/adapter_cook_assistant_1"
os.makedirs(ADAPTER_DIR, exist_ok=True)

# trainer.model is the PEFT-wrapped model after training
trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)

print("Adapter saved to:", ADAPTER_DIR)
# Check it exists
import glob
print("Adapter files:", glob.glob(os.path.join(ADAPTER_DIR, "*")))


Adapter saved to: ./results/adapter_cook_assistant_1
Adapter files: ['./results/adapter_cook_assistant_1/README.md', './results/adapter_cook_assistant_1/adapter_model.safetensors', './results/adapter_cook_assistant_1/adapter_config.json', './results/adapter_cook_assistant_1/chat_template.jinja', './results/adapter_cook_assistant_1/tokenizer_config.json', './results/adapter_cook_assistant_1/special_tokens_map.json', './results/adapter_cook_assistant_1/added_tokens.json', './results/adapter_cook_assistant_1/vocab.json', './results/adapter_cook_assistant_1/merges.txt', './results/adapter_cook_assistant_1/tokenizer.json']


In [35]:
# 2) MERGE USING CLEAN BASE + SAVED ADAPTER
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen3-0.6B"
ADAPTER_DIR = "./results/adapter_cook_assistant_1"
MERGED_DIR = "./results/merged_cook_assistant_1"

# Load clean base (no adapters attached)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True, use_fast=True)

# Attach the single saved adapter
model_with_adapter = PeftModel.from_pretrained(base_model, ADAPTER_DIR)

# Merge LoRA weights into the base model and unload PEFT wrapper
merged_model = model_with_adapter.merge_and_unload()

# Save merged full model
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("Merged full model saved to:", MERGED_DIR)

# Inspect contents
import glob, os
print("Merged files:", glob.glob(os.path.join(MERGED_DIR, "*")))


Merged full model saved to: ./results/merged_cook_assistant_1
Merged files: ['./results/merged_cook_assistant_1/README.md', './results/merged_cook_assistant_1/adapter_model.safetensors', './results/merged_cook_assistant_1/adapter_config.json', './results/merged_cook_assistant_1/chat_template.jinja', './results/merged_cook_assistant_1/tokenizer_config.json', './results/merged_cook_assistant_1/special_tokens_map.json', './results/merged_cook_assistant_1/added_tokens.json', './results/merged_cook_assistant_1/vocab.json', './results/merged_cook_assistant_1/merges.txt', './results/merged_cook_assistant_1/tokenizer.json', './results/merged_cook_assistant_1/config.json', './results/merged_cook_assistant_1/generation_config.json', './results/merged_cook_assistant_1/model.safetensors']


In [36]:
import os
from huggingface_hub import HfApi

hf_token = os.getenv("HF_TOKEN")
api = HfApi(token=hf_token)

In [37]:
from huggingface_hub import create_repo

merged_repo  = "anileo1/cook-assistant-Qwen3-0.6B"
create_repo(merged_repo, repo_type="model", private=False, exist_ok=True, token=hf_token)

RepoUrl('https://huggingface.co/anileo1/cook-assistant-Qwen3-0.6B', endpoint='https://huggingface.co', repo_type='model', repo_id='anileo1/cook-assistant-Qwen3-0.6B')

In [38]:
from huggingface_hub import upload_folder

merged_dir  = "./results/merged_cook_assistant_1"

# Upload merged full model
upload_folder(
    repo_id=merged_repo,
    folder_path=merged_dir,
    repo_type="model",
    token=hf_token,
    commit_message="Upload merged full model: Cook Assistant v1",
)

print(f"Uploaded:\n- Merged  -> {merged_repo}")


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploaded:
- Merged  -> anileo1/cook-assistant-Qwen3-0.6B


In [31]:
# 1) Reload a clean base model and attach the single trained adapter, then merge
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen3-0.6B"  # your base
ADAPTER_DIR = "./results/merged_cook_assistant_1"  # where you saved adapter earlier
MERGED_DIR = "./results/merged_cook_assistant_1"

# Load clean base (no adapters)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True, use_fast=True)

# Attach the adapter once
model_with_adapter = PeftModel.from_pretrained(base_model, ADAPTER_DIR)

# Merge LoRA weights into base and unload PEFT wrapper
merged_model = model_with_adapter.merge_and_unload()

# Save the merged full model
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("Saved merged full model to:", MERGED_DIR)


`torch_dtype` is deprecated! Use `dtype` instead!


ValueError: Can't find 'adapter_config.json' at './results/adapter_cook_assistant_1'

In [40]:
# Inference with merged model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MERGED_DIR = "./results/merged_cook_assistant_1"

# Load merged model (plain HF model) and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MERGED_DIR, trust_remote_code=True, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(MERGED_DIR, dtype=torch.bfloat16)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Build a chat prompt using the model's chat template
messages = [
    {"role": "system", "content": "You are a helpful assistant that generates recipe samples from a given set of ingredients."},
    {"role": "user", "content": "Generate a recipe using the following ingredients: tomatoes, basil, garlic, olive oil, pasta. Include a title and clear steps."},
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,  # add assistant turn for generation
)

inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.inference_mode():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=8192,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

# Decode and print only the generated assistant part
generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated)


Loading adapter weights from ./results/merged_cook_assistant_1 led to missing keys in the model: model.layers.0.self_attn.q_proj.lora_A.default.weight, model.layers.0.self_attn.q_proj.lora_B.default.weight, model.layers.0.self_attn.v_proj.lora_A.default.weight, model.layers.0.self_attn.v_proj.lora_B.default.weight, model.layers.1.self_attn.q_proj.lora_A.default.weight, model.layers.1.self_attn.q_proj.lora_B.default.weight, model.layers.1.self_attn.v_proj.lora_A.default.weight, model.layers.1.self_attn.v_proj.lora_B.default.weight, model.layers.2.self_attn.q_proj.lora_A.default.weight, model.layers.2.self_attn.q_proj.lora_B.default.weight, model.layers.2.self_attn.v_proj.lora_A.default.weight, model.layers.2.self_attn.v_proj.lora_B.default.weight, model.layers.3.self_attn.q_proj.lora_A.default.weight, model.layers.3.self_attn.q_proj.lora_B.default.weight, model.layers.3.self_attn.v_proj.lora_A.default.weight, model.layers.3.self_attn.v_proj.lora_B.default.weight, model.layers.4.self_att

system
You are a helpful assistant that generates recipe samples from a given set of ingredients.
user
Generate a recipe using the following ingredients: tomatoes, basil, garlic, olive oil, pasta. Include a title and clear steps.
assistant
<think>
Okay, let's see. The user wants a recipe using tomatoes, basil, garlic, olive oil, and pasta. They also specified including a title and clear steps. Alright.

First, I need to come up with a catchy title. Maybe something like "Simple Tomato Pasta with Basil Garlic" or "Garlic-Infused Tomato Pasta". That should grab attention.

Next, the steps. Let me think about a classic tomato pasta dish. Typically, you mix fresh tomatoes, add garlic and basil, maybe some olive oil, then cook them with pasta. Wait, but sometimes you use whole eggs. But since the ingredients include olive oil, maybe just the olive oil? Or maybe combine it with the pasta. Hmm.

Wait, the user provided only the ingredients. So the steps should be based on those. So step one co